<a href="https://colab.research.google.com/github/mwahaha-umich/ACLUFinalProject/blob/main/Data_Prep_Anupriya_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Jupyter notebook is intended to assist with accessing plotly from outside of a Google Colab environment (Should technically work for any cloud environment that allows you to install NGROK). The problem with running plotly within Google Colab is that the ports that are opened by Plotly are not exposed to the outside world. In order to expose them we are going to create a network tunnel, that's where NGROK comes in.

A few things of note:
*  If you are using Google Drive for any information for this plot, you will want to copy that data "locally" before the graph will fuction externally. This is because we are starting up a new thread that will not carry over the Google Driver permissions you may have granted to the current thread.


# Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import requests

if 'google.colab' in str(get_ipython()):
  #We keep changing the different paths, so I'm going to check if various configurations exist.
  if (os.path.isdir("/content/drive/MyDrive/Shared with me/content/drive/My Drive/Shared with me/ACLU/")):
    GoogleDriveBase = "/content/drive/MyDrive/Shared with me/content/drive/My Drive/Shared with me/" #Anupriya
    WorkingDirectory = GoogleDriveBase + 'ACLU/' #Anupriya
  else:  
    if (os.path.isdir("/content/drive/My Drive/Projects/ACLU/")):
      GoogleDriveBase = "/content/drive/My Drive/" #Kyle
      WorkingDirectory = GoogleDriveBase + "Projects/ACLU/" #Kyle
else: # We're not running in Google Colab, which means we're probably running locally. 
  #Put code here for local copies of the files
  GoogleDriveBase = "" 
  WorkingDirectory = GoogleDriveBase + "" 


WorkingFiles = WorkingDirectory + 'WorkingFiles/'
BasePickeDrive = GoogleDriveBase + WorkingDirectory + "Pickle/"


#Make the necessary folders for the script to run.
ListOfAllRequiredDirectories = [WorkingDirectory + 'Pickle', 
                          WorkingDirectory + 'WorkingFiles',
                          WorkingDirectory + 'AdditionalData',
                          WorkingDirectory + 'ACLUData',
                          WorkingDirectory + 'DashboardFiles']

for folder in ListOfAllRequiredDirectories:
  RunningPath = GoogleDriveBase + folder + "/"
  Path(RunningPath).mkdir(parents=True, exist_ok=True)

In [3]:
import pandas as pd
import math
import numpy as np

df = pd.read_csv(WorkingFiles + "dfVoters.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,28,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
dfVoters_sub = df[['Voter File VANID', 'FirstName', 'MiddleName',
       'LastName', 'DWID', 'Zip5', 'Age', 
       'MaritalStatus', 'Sex', 'City','State', 'Zip4', 'Suffix', 'CD', 'SD', 'HD', 'CountyName', 'DOB', 'DateReg',
       'EthnicCatalistName', 'Party', 'RaceName', 'Primary19', '2020_Biden_Support', 'Voting_Aug_Prim',
       'PoliceAccountability','VBM_Application', 'MarijuanaConviction','Absentee_Voting', 'Mass_Incarceration']]

bins= [18,29,44,64, 123]
labels = ['18-29','30-44','45-64','65+']
dfVoters_sub['AgeGroup'] = pd.cut(dfVoters_sub['Age'], bins=bins, labels=labels, right=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [5]:
#Crashing for Memory issue
import pandas as pd
dfreshapedUnique = pd.read_csv(WorkingFiles + 'dfreshapedUnique.csv')
dfreshapedUnique = dfreshapedUnique[['Voter File VANID', 'year', 'electiontype']]

election_votes = pd.merge(dfreshapedUnique, dfVoters_sub, how='left', on='Voter File VANID')

election_votes = election_votes[['Voter File VANID', 'year', 'electiontype', 
                                'CountyName', 'AgeGroup', 'RaceName', 'Sex']]

election_votes['CountyName'] = election_votes['CountyName'].astype("str").replace(np.nan, 'nan', regex=True)
election_votes['AgeGroup'] = election_votes['AgeGroup'].astype("str").replace(np.nan, 'nan', regex=True)
election_votes['RaceName'] = election_votes['RaceName'].astype("str").replace(np.nan, 'nan', regex=True)
election_votes['Sex'] = election_votes['Sex'].astype("str").replace(np.nan, 'nan', regex=True)

print(len(election_votes))
election_votes = election_votes.groupby(['year', 'electiontype', 
                                'CountyName', 'AgeGroup', 'RaceName', 'Sex']).count().reset_index()
print(len(election_votes))
election_votes = election_votes.rename(columns={"Voter File VANID": "Count"})

election_votes.to_csv(WorkingDirectory + 'WorkingFiles/' + "election_votes.csv")
print(election_votes.head())
del election_votes
del dfreshapedUnique

16125018
39599
   year electiontype CountyName AgeGroup   RaceName Sex  Count
0  1998    General##     Alcona    45-64  Caucasian   M      1
1  1998    General##      Alger    45-64  Caucasian   F      1
2  1998    General##      Alger      65+  Caucasian   M      1
3  1998    General##    Allegan    45-64  Caucasian   F      9
4  1998    General##    Allegan    45-64  Caucasian   M      6


If you would like to run the previous example externally, just copy and paste your code into a new block like below. 

## Folium Map Data Prep

In [6]:
registered_voters = pd.read_csv(WorkingDirectory + 'AdditionalData/All Registered Voters Michigan.csv')
registered_voters.rename(columns = {'County':'CountyName'}, inplace = True)
county_pop_df = dfVoters_sub.groupby('CountyName').count()['Voter File VANID'].to_frame().reset_index()
county_pop_df['Voter File VANID'] = county_pop_df['Voter File VANID'].fillna(0)
county_pop_df['CountyName'] = county_pop_df['CountyName'].str.upper()


registered_voters.columns

total_voter_pop = pd.merge(county_pop_df, registered_voters, how = 'left', on = 'CountyName')
total_voter_pop['Voter File VANID'].astype(int)
print(total_voter_pop[pd.isna(total_voter_pop['All Registered Voters'])])

total_voter_pop['All Registered Voters'].astype(int)
total_voter_pop['fraction'] = total_voter_pop['Voter File VANID'] / total_voter_pop['All Registered Voters']
total_voter_pop['CountyName'] = total_voter_pop['CountyName'].str.title()
total_voter_pop['RaceName'] = 'All'
total_voter_pop['Sex'] = 'All'
total_voter_pop['AgeGroup'] = 'All'

#Save this to CSV
total_voter_pop.to_csv(WorkingDirectory + 'WorkingFiles/' + "total_voter_pop.csv")
del total_voter_pop

Empty DataFrame
Columns: [CountyName, Voter File VANID, Number, Feb Active Registered Voters, All Registered Voters, July Active Registered Voters, July All Registered Voters, October Active registered Voters, October All Registered Voters]
Index: []


## Try Map w Plotly 

We can see that the above graph does not work very well with just joining on name because there could be many counties with the same name. So we are going to filter the json to get just the state of Michigan and use the FIPS IDs to just make it really clean. 

In [7]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

counties["features"][0]

{'geometry': {'coordinates': [[[-86.496774, 32.344437],
    [-86.717897, 32.402814],
    [-86.814912, 32.340803],
    [-86.890581, 32.502974],
    [-86.917595, 32.664169],
    [-86.71339, 32.661732],
    [-86.714219, 32.705694],
    [-86.413116, 32.707386],
    [-86.411172, 32.409937],
    [-86.496774, 32.344437]]],
  'type': 'Polygon'},
 'id': '01001',
 'properties': {'CENSUSAREA': 594.436,
  'COUNTY': '001',
  'GEO_ID': '0500000US01001',
  'LSAD': 'County',
  'NAME': 'Autauga',
  'STATE': '01'},
 'type': 'Feature'}

In [8]:
data = {}
nest1 = counties["features"]
for thing in nest1: 
  if thing['properties']['STATE'] == '26':
    data[thing['id']] = thing['properties']['NAME']

In [9]:
trial = pd.DataFrame(data, index=[0])
mi_fips = trial.T.reset_index()
mi_fips = mi_fips.rename(columns={"index": "FIPS", 0: 'CountyName'})
mi_fips.head()

,FIPS,CountyName
0,26009,Antrim
1,26015,Barry
2,26023,Branch
3,26035,Clare
4,26037,Clinton


In [10]:
registered_voters = pd.read_csv(WorkingDirectory + 'AdditionalData/All Registered Voters Michigan.csv')
registered_voters.rename(columns = {'County':'CountyName'}, inplace = True)
county_pop_df = dfVoters_sub.groupby('CountyName').count()['Voter File VANID'].to_frame().reset_index()
county_pop_df['Voter File VANID'] = county_pop_df['Voter File VANID'].fillna(0)
county_pop_df['CountyName'] = county_pop_df['CountyName'].str.upper()

total_voter_pop = pd.merge(county_pop_df, registered_voters, how = 'left', on = 'CountyName')
total_voter_pop['Voter File VANID'].astype(int)
print(total_voter_pop[pd.isna(total_voter_pop['All Registered Voters'])])

total_voter_pop['All Registered Voters'].astype(int)
total_voter_pop['fraction'] = total_voter_pop['Voter File VANID'] / total_voter_pop['All Registered Voters']
total_voter_pop['CountyName'] = total_voter_pop['CountyName'].str.title()
total_voter_pop['RaceName'] = 'All'
total_voter_pop['Sex'] = 'All'
total_voter_pop['AgeGroup'] = 'All'
total_voter_pop.head()

plotly_fips = pd.merge(total_voter_pop, mi_fips, how = 'left', on = 'CountyName')

plotly_fips.to_csv(WorkingDirectory + 'WorkingFiles/' + "plotly_fips.csv")

del county_pop_df
del registered_voters
del total_voter_pop
del plotly_fips

Empty DataFrame
Columns: [CountyName, Voter File VANID, Number, Feb Active Registered Voters, All Registered Voters, July Active Registered Voters, July All Registered Voters, October Active registered Voters, October All Registered Voters]
Index: []


## Biden Support

In [11]:
biden_support_df = dfVoters_sub.groupby(["RaceName", "Sex", "AgeGroup", "CountyName", '2020_Biden_Support']).count()['Voter File VANID'].to_frame().reset_index()
biden_support_df['Voter File VANID'] = biden_support_df['Voter File VANID'].fillna(0)
biden_support_df = biden_support_df.rename(columns={"Voter File VANID": "# of Voters"})
biden_support_df["2020_Biden_Support"] = biden_support_df["2020_Biden_Support"].str.replace('\d+\.\s','')
biden_support_df.head()

biden_support_df.to_csv(WorkingDirectory + 'WorkingFiles/'  + "biden_support_df.csv")
del biden_support_df

## Pie Charts

In [12]:
mass_inc = dfVoters_sub.groupby(["RaceName", "Sex", "AgeGroup", "CountyName",'Mass_Incarceration']).count()['Voter File VANID'].to_frame().reset_index()
mass_inc = mass_inc.rename(columns={"Voter File VANID": "Count"})

policing = dfVoters_sub.groupby(["RaceName", "Sex", "AgeGroup", "CountyName",'PoliceAccountability']).count()['Voter File VANID'].to_frame().reset_index()
policing = policing.rename(columns={"Voter File VANID": "Count"})

weed = dfVoters_sub.groupby(["RaceName", "Sex", "AgeGroup", "CountyName",'MarijuanaConviction']).count()['Voter File VANID'].to_frame().reset_index()
weed = weed.rename(columns={"Voter File VANID": "Count"})


mass_inc.to_csv(WorkingDirectory + 'WorkingFiles/'  + "mass_inc.csv")
policing.to_csv(WorkingDirectory + 'WorkingFiles/'  + "policing.csv")
weed.to_csv(WorkingDirectory + 'WorkingFiles/'  + "weed.csv")

del mass_inc
del policing
del weed
#Not delting dfVoters_sub as it's needed in a moment

## Election Line Chart

# Build our Dashboard

In [32]:
#Set up all our filter columns
FilterColumnNames = ["RaceName", "Sex", "AgeGroup", "CountyName"]
FilterColumns = {}
import math
import numpy as np
import pickle

for column in FilterColumnNames:
  FilterColumns[column] = list(dfVoters_sub[column].astype("str").replace(np.nan, 'nan', regex=True).unique())
  FilterColumns[column].append("All")
  print(FilterColumns[column])

with open(WorkingDirectory + 'WorkingFiles/' + 'FilterColumns.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(FilterColumns, filehandle)

ImportantColumnNames = FilterColumnNames.copy()
ImportantColumnNames.append(['CommunicationType', 'ElectionType'])
dfOutreach = pd.read_csv(WorkingFiles + "dfOutreach.csv")
outreach_voters = pd.merge(dfOutreach, dfVoters_sub, how='left', on='Voter File VANID')

outreach_dem_df = outreach_voters[['Voter File VANID', 'ElectionType', 'CommunicationType',
                                   'CountyName', 'AgeGroup', 'RaceName', 'Sex']]

outreach_dem_df_nan = outreach_dem_df.copy()
outreach_dem_df_nan["AgeGroup"] = outreach_dem_df_nan["AgeGroup"].astype("str")
outreach_dem_df_nan.head()

outreach_dem_df_nan = outreach_dem_df_nan.replace(np.nan, 'nan', regex=True)
outreach_dem_df_nan = outreach_dem_df_nan.replace(np.nan, 'nan', regex=True)


df_aggregate = outreach_dem_df_nan.groupby(["RaceName", "Sex", "AgeGroup", "CountyName", 
                                        'CommunicationType', 'ElectionType']).count().reset_index()
df_aggregate = df_aggregate.rename(columns={"Voter File VANID": "Count", 'ElectionType': 'CampaignType'})

df_aggregate['CampaignType'] = df_aggregate['CampaignType'].replace({'lpv': 'Statewide voting rights/voter education race', 
                                                           'scomi': 'Supreme Court of Michigan Race',
                                                           'oakland': 'Oakland County Prosecutor Race', 
                                                           'oakland prosecutor': 'Oakland County Prosecutor Race', 
                                                           'oakland county prosecutor race': 'Oakland County Prosecutor Race'})

df_aggregate.to_csv(WorkingDirectory + 'WorkingFiles/' + "df_aggregate.csv")
del df_aggregate

['Caucasian', 'Asian', 'Black', 'Unknown', 'Hispanic', 'Native American', 'All']
['M', 'F', 'U', 'All']
['65+', '45-64', '18-29', '30-44', 'nan', 'All']
['Macomb', 'Kent', 'Washtenaw', 'Wayne', 'Genesee', 'Ingham', 'Saginaw', 'Oakland', 'Calhoun', 'Kalamazoo', 'Isabella', 'Livingston', 'Ottawa', 'St. Joseph', 'Muskegon', 'Midland', 'Allegan', 'Emmet', 'St. Clair', 'Berrien', 'Marquette', 'Bay', 'Alpena', 'Eaton', 'Monroe', 'Shiawassee', 'Jackson', 'Van Buren', 'Mackinac', 'Lapeer', 'Osceola', 'Clinton', 'Menominee', 'Presque Isle', 'Wexford', 'Houghton', 'Montmorency', 'Lenawee', 'Grand Traverse', 'Dickinson', 'Gratiot', 'Kalkaska', 'Iosco', 'Antrim', 'Montcalm', 'Mason', 'Ionia', 'Barry', 'Tuscola', 'Manistee', 'Newaygo', 'Leelanau', 'Oceana', 'Mecosta', 'Delta', 'Oscoda', 'Arenac', 'Huron', 'Iron', 'Sanilac', 'Branch', 'Cheboygan', 'Luce', 'Gogebic', 'Otsego', 'Ogemaw', 'Lake', 'Hillsdale', 'Chippewa', 'Cass', 'Baraga', 'Gladwin', 'Charlevoix', 'Crawford', 'Benzie', 'Schoolcraft', 'R

Now for the final few bits before we can run this properly

In [23]:
# How to run a Dash app in Google Colab

## Requirements

### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip


get_ipython().system_raw('./ngrok http 8050 &')

### Install Dash
!pip install dash  # The core dash backend
!pip install jupyter_dash
!pip install -U plotly
#Do not install any of these. When you install Dash, it includes these. If you call the install it installs a previous version of these and will generate an error.
#!pip install dash-html-components  # HTML components
#!pip install dash-core-components  # Supercharged components
#!pip install dash-table  # Interactive DataTable component (new!)

--2021-07-26 16:05:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.44.39.24, 3.228.26.119, 3.228.119.185, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.44.39.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  5.94MB/s    in 2.2s    

2021-07-26 16:06:02 (5.94 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
     |████████████████████████████████| 1.1 MB 4.1 MB/s 
     |████████████████████████████████| 3.7 MB 14.4 MB/s 
     |████████████████████████████████| 83 kB 2.2 MB/s 
     |████████████████████████████████| 1.8 MB 42.2 MB/s 
     |████████████████████████████████| 357 kB 49.9 MB/s 
  Created wheel for dash: filename=dash-1.21.0-py3-none-any.whl 

In [45]:
#Copy out files
from shutil import copyfile
copyfile(WorkingDirectory + 'WorkingFiles/' + "df_aggregate.csv", "df_aggregate.csv")
copyfile(WorkingDirectory + 'WorkingFiles/' + "total_voter_pop.csv", "total_voter_pop.csv")
copyfile(WorkingDirectory + 'WorkingFiles/' + "biden_support_df.csv", "biden_support_df.csv")
copyfile(WorkingDirectory + 'WorkingFiles/' + "mass_inc.csv", "mass_inc.csv")
copyfile(WorkingDirectory + 'WorkingFiles/' + "policing.csv", "policing.csv")
copyfile(WorkingDirectory + 'WorkingFiles/' + "weed.csv", "weed.csv")
copyfile(WorkingDirectory + 'WorkingFiles/' + "election_votes.csv", "election_votes.csv")
copyfile(WorkingDirectory + 'WorkingFiles/' + "plotly_fips.csv", "plotly_fips.csv")
copyfile(WorkingDirectory + 'WorkingFiles/' + "FilterColumns.data", "FilterColumns.data")
copyfile(WorkingDirectory + 'AdditionalData/' + "Counties_(v17a).geojson", "Counties_(v17a).geojson")



'Counties_(v17a).geojson'

In [60]:
import plotly.express as px
election_votes = pd.read_csv("election_votes.csv")
election_votes = election_votes.groupby(['year', 'electiontype']).sum().reset_index()
px.line(
          election_votes, 
          x='year', 
          y='Count', 
          color = 'electiontype')




In [64]:
%%writefile AltairGraph.py

import altair as alt
import io
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from jupyter_dash import JupyterDash
import plotly.express as px
from vega_datasets import data
import folium
import pandas as pd
import pickle
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

with open('FilterColumns.data', 'rb') as filehandle:
    # read the data as binary data stream
    FilterColumns = pickle.load(filehandle)

df_aggregate = pd.read_csv("df_aggregate.csv")
total_voter_pop = pd.read_csv("total_voter_pop.csv")
biden_support_df = pd.read_csv("biden_support_df.csv")
mass_inc = pd.read_csv("mass_inc.csv")
policing = pd.read_csv("policing.csv")
weed = pd.read_csv("weed.csv")
election_votes = pd.read_csv("election_votes.csv")
plotly_fips = pd.read_csv("plotly_fips.csv")
#alt.data_transformers.disable_max_rows()

# Don't need this with the cars dataset
alt.data_transformers.enable('default', max_rows=150000)

cars = data.cars()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.css.append_css({'external_url':
                    'https://cdn.rawgit.com/gschivley/8040fc3c7e11d2a4e7f0589ffc829a02/raw/fe763af6be3fc79eca341b04cd641124de6f6f0d/dash.css'
                    })
app.title = 'Test dash and altair'
server = app.server

#Build our dropdowns for each of the Filter Columns
DropDowns = []
inputs = []
for ColumnName in FilterColumns.keys():
    DropDowns.append(
        html.Div([
            html.Label(ColumnName),
            dcc.Dropdown(
                id=ColumnName,
                options=[{'label': i, 'value':i} for i in FilterColumns[ColumnName]],
                value='All'
            )
        ],
        style={'width': '250px', 'marginRight': 'auto',
              'marginLeft': 'auto', 'textAlign': 'center'},
        className='column'))
    inputs.append(dash.dependencies.Input(ColumnName, 'value'))

#Build our main layout
app.layout = html.Div(children=[html.Div(children=DropDowns, className='row'), html.Pre(id='plot', children="If this doesn't go away you have an error")])

@app.callback(
    dash.dependencies.Output('plot', 'children'),
    inputs
)
def pick_figure(Race, Sex, AgeGroup, CountyName):
    #This is our key. Filter everything before passing it into the charts.
    dfFiltered = df_aggregate[((df_aggregate['RaceName'] == Race) | (Race == 'All') ) & 
                              ((df_aggregate['Sex'] == Sex) | (Sex == 'All')) &
                              ((df_aggregate['AgeGroup'] == AgeGroup) | (AgeGroup == 'All')) &
                              ((df_aggregate['CountyName'] == CountyName) | (CountyName == 'All'))]
    
    total_voter_pop_filtered = total_voter_pop[((total_voter_pop['RaceName'] == Race) | (Race == 'All') ) & 
                              ((total_voter_pop['Sex'] == Sex) | (Sex == 'All')) &
                              ((total_voter_pop['AgeGroup'] == AgeGroup) | (AgeGroup == 'All')) &
                              ((total_voter_pop['CountyName'] == CountyName) | (CountyName == 'All'))]
    
    biden_support_filtered = biden_support_df[((biden_support_df['RaceName'] == Race) | (Race == 'All') ) & 
                              ((biden_support_df['Sex'] == Sex) | (Sex == 'All')) &
                              ((biden_support_df['AgeGroup'] == AgeGroup) | (AgeGroup == 'All')) &
                              ((biden_support_df['CountyName'] == CountyName) | (CountyName == 'All'))]
    
    mass_inc_filtered = mass_inc[((mass_inc['RaceName'] == Race) | (Race == 'All') ) & 
                              ((mass_inc['Sex'] == Sex) | (Sex == 'All')) &
                              ((mass_inc['AgeGroup'] == AgeGroup) | (AgeGroup == 'All')) &
                              ((mass_inc['CountyName'] == CountyName) | (CountyName == 'All'))]
    
    policing_filtered = policing[((policing['RaceName'] == Race) | (Race == 'All') ) & 
                              ((policing['Sex'] == Sex) | (Sex == 'All')) &
                              ((policing['AgeGroup'] == AgeGroup) | (AgeGroup == 'All')) &
                              ((policing['CountyName'] == CountyName) | (CountyName == 'All'))]
    
    weed_filtered = weed[((weed['RaceName'] == Race) | (Race == 'All') ) & 
                              ((weed['Sex'] == Sex) | (Sex == 'All')) &
                              ((weed['AgeGroup'] == AgeGroup) | (AgeGroup == 'All')) &
                              ((weed['CountyName'] == CountyName) | (CountyName == 'All'))]
    
    election_votes_filtered = election_votes[((election_votes['RaceName'] == Race) | (Race == 'All') ) & 
                              ((election_votes['Sex'] == Sex) | (Sex == 'All')) &
                              ((election_votes['AgeGroup'] == AgeGroup) | (AgeGroup == 'All')) &
                              ((election_votes['CountyName'] == CountyName) | (CountyName == 'All'))]
    election_votes_filtered = election_votes_filtered.groupby(['year', 'electiontype']).sum().reset_index()
    
    plotly_fips_filtered = plotly_fips[((plotly_fips['RaceName'] == Race) | (Race == 'All') ) & 
                              ((plotly_fips['Sex'] == Sex) | (Sex == 'All')) &
                              ((plotly_fips['AgeGroup'] == AgeGroup) | (AgeGroup == 'All')) &
                              ((plotly_fips['CountyName'] == CountyName) | (CountyName == 'All'))]
    
    
    

    #We should also aggregate it. This will keep our record count low and avoid errors. We will obviously need to add columns that we need.
    #dfFiltered = dfFiltered.groupby(by=['2020_Biden_Support', 'CountyName']).sum()
    #dfFiltered = dfFiltered.reset_index()

    #Encode the Altair Chart
    base = alt.Chart(dfFiltered).mark_bar().encode(
    x='Count:Q',
    y = alt.Y('CommunicationType:N', title=None),
    color='CommunicationType:N',
    row='ElectionType:N').properties(title='Election by Communication Type')

    # Save html as a StringIO object in memory
    htmloutput = io.StringIO()
    base.save(htmloutput, 'html')

    chart1 = html.Iframe(
        height='500',
        width='1000',
        #sandbox='allow-scripts',

        # This is where we will pass the html
        srcDoc = htmloutput.getvalue(),

        # Get rid of the border box
        style={'border-width': '0px'}
    )

    #Create a simple Plotly Express chart

    chart2 = dcc.Graph(
      id='example-graph',
      figure=px.histogram(
          biden_support_filtered, 
          x="2020_Biden_Support", 
          y="# of Voters", 
          barmode='group', histfunc='sum',
          title="Biden 2020 Support", 
          labels={'2020_Biden_Support':''}),
      style={'width': '100vh', 'height': '50vh', 'float' : 'right','display': 'inline-block' }
    )

    #Add a folium map
    #start_coords = (44.3148, -85.6024)
    #folium_map = folium.Map(location=start_coords, tiles='cartodbpositron', zoom_start=6)
    miMap = folium.Map(location=[44.3148, -85.6024], tiles='cartodbpositron', zoom_start=7)
    m = folium.Choropleth(geo_data='Counties_(v17a).geojson',
                 data=total_voter_pop_filtered, columns=['CountyName', 'fraction'], key_on='feature.properties.NAME',
                 fill_color='YlGnBu', fill_opacity=1).add_to(miMap)

    
    chart3 = html.Iframe(
        height='2000',
        width='1000',

        # This is where we will pass the html
        srcDoc = miMap._repr_html_(),

        # Get rid of the border box
        style={'border-width': '0px', 'float': 'right'}
    )
    
    chart4 = dcc.Graph(
      id='mass-inc-graph',
      figure=px.pie(
          mass_inc_filtered, 
          values="Count",
          names="Mass_Incarceration", 
          title="Mass Incarceration", 
          labels={'Mass_Incarceration':'Support'}, 
          color='Mass_Incarceration',
          color_discrete_map={'Yes':'lightgreen',
                                 'No':'crimson',
                                 'Other/Unknown':'cyan'}),
      style={'width': '50vh', 'height': '50vh', 'float': 'right'}
    )
    
    chart5 = dcc.Graph(
      id='policing-graph',
      figure=px.pie(
          policing_filtered, 
          values="Count",
          names="PoliceAccountability", 
          title="Police Accountability", 
          labels={'PoliceAccountability':'Support'}, 
          color='PoliceAccountability',
          color_discrete_map={'Yes':'lightgreen',
                                 'No':'crimson',
                                 'Other/Unknown':'cyan',
                                 'Maybe':'goldenrod'}),
      style={'width': '50vh', 'height': '50vh', 'float': 'right'}
    )

    chart6 = dcc.Graph(
      id='weed-graph',
      figure=px.pie(
          weed_filtered, 
          values="Count",
          names="MarijuanaConviction", 
          title="Marijuana Conviction", 
          labels={'Marijuana Conviction':'Support'}, 
          color='MarijuanaConviction',
          color_discrete_map={'Yes':'lightgreen',
                                 'No':'crimson',
                                 'Other/Unknown':'cyan'}),
      style={'width': '50vh', 'height': '50vh', 'float': 'right'}
    )

    chart7 = dcc.Graph(
      id='election-graph',
      figure=px.line(
          election_votes_filtered, 
          x='year', 
          y='Count', 
          color = 'electiontype'),
      style={'width': '200vh', 'height': '100vh', 'float': 'right', 'display': 'inline-block'}
    )

    plotly_map = dcc.Graph(
      id='map-graph',
      figure=px.choropleth(plotly_fips_filtered, geojson=counties, locations='FIPS', color='fraction',
                           color_continuous_scale="Viridis_r",
                           range_color=(0, 0.3),
                           scope="usa",
                           labels={'fraction':'ACLU Voters by Registered Voters'}, 
                    hover_data=["CountyName", 'fraction', 'Voter File VANID', 'All Registered Voters']
                          ).update_geos(fitbounds="locations", visible=False).update_layout(margin={"r":0,"t":0,"l":0,"b":0}),
      style={'width': '200vh', 'height': '100vh', 'float': 'right'}
    )

    comm_graph = dcc.Graph(
      id='comm-graph',
      figure=px.histogram(df_aggregate, x="CommunicationType", y="Count", color="CommunicationType", 
                          barmode="group", histfunc='sum', facet_col="CampaignType", facet_col_spacing=0.05, 
                          title="Campaign by Communication Type").for_each_annotation(lambda a: a.update(text=a.text.split("=")[1])),
      style={'width': '150vh', 'height': '50vh', 'float': 'right'}
    )

    
    # Return the html from StringIO object
    return plotly_map, comm_graph, chart4, chart2, chart5, chart6, chart7
    #return chart2

if __name__ == '__main__':
    app.run_server(debug=True)

#app.run_server(mode="inline")

Overwriting AltairGraph.py


In [37]:
import os
import json
import time

def GetPublicIP(NumberOfLoops):
  #This code only works on a linux based OS, we'll skip running it if we're on an unknown OS
  #if os.name in LinuxOSList:
  NumberOfLoops = NumberOfLoops + 1
  get_ipython().system_raw('./ngrok http 8050 &')
  time.sleep(1)
  result = os.popen("curl -s http://localhost:4040/api/tunnels").read()
  print(result)
  resultDict = json.loads(result)
  if 'tunnels' in resultDict.keys():
    publicIP = resultDict['tunnels'][0]['public_url']
    print(publicIP)
    return publicIP
  else:
    print("Number of Loops: " + NumberOfLoops)
    if NumberOfLoops > 5:
      return "Unable to get public IP"
    return GetPublicIP(NumberOfLoops)

In [65]:
publicIP = GetPublicIP(5)
### Run Dash app
!python AltairGraph.py

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://e499b0136631.ngrok.io","proto":"https","config":{"addr":"http://localhost:8050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://e499b0136631.ngrok.io","proto":"http","config":{"addr":"http://localhost:8050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}

https://e499b0136631.ngrok.io
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "AltairGraph" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
/usr/local/